In [2]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
db = client.apan5400

In [43]:
import json

json_data = open("companies.json").read()

newsfeeds = []
json_data = (json.loads(json_data))
for line in json_data:
    newsfeeds.append(line)
print(len(newsfeeds))

15


In [44]:
print(newsfeeds)
collection = db.companies
collection.insert_many(newsfeeds)

[{'company': 'Apple', 'city': 'Cupertino', 'state': 'California', 'country': 'USA', 'description': 'American multinational technology company that designs, develops, and sells consumer electronics, computer software, and online services', 'employees': 154000, 'founded': 1976, 'revenue': 365.82}, {'company': 'Microsoft', 'city': 'Redmond', 'state': 'Washington', 'country': 'USA', 'description': 'American multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services.', 'employees': 182268, 'founded': 1975, 'revenue': 161}, {'company': 'Amazon', 'city': 'Seattle', 'state': 'Washington', 'country': 'USA', 'description': 'American multinational technology company which focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence.', 'employees': 1608000, 'founded': 1994, 'revenue': 469.83}, {'company': 'Alphabet', 'city': 'Mountain View', 'state': 'Califor

In [47]:
total_docs = collection.count_documents({})
total_docs

15

# 1.Print the list of companies established in the 20th century in the descending order of the number of employees.

In [141]:
query1 = {
    "founded": {
        "$gte": 1900,
        "$lt": 2000,
    }
}

results = collection.find(query1).sort("employees",-1)
case = collection.count_documents(query1)
for item in results[:10]:
    print(item['company'])

Amazon
China National Petroleum Corporation
Toyota Motor
Mercedes-Benz Group
Samsung Electronics
Alibaba Group
Microsoft
Alphabet
Apple
Glencore


# 2. Print the totals of annual revenues of the companies headquartered in the USA and Germany, respectively.

In [147]:
pipeline1 = [
    {'$match': {'country':{'$regex':'(USA)|(Germany)',
                           "$options": 'i'}}},
    {'$group': {'_id': "$country", 'revenue_sum':{'$sum': "$revenue"}}}
]
list(collection.aggregate(pipeline1))
#results = collection.aggregate(pipeline1)
#print(results)


[{'_id': 'USA', 'revenue_sum': 1308.09},
 {'_id': 'Germany', 'revenue_sum': 334.88}]

# 3. Print the city where the company that has the world's second largest proven crude oil reserves headquartered.

In [153]:
collection.create_index([('description', 'text')])
query2= {
    "$text" :{'$search': "the world's second largest proven crude oil reserves"}
}

results = collection.find(query2)
print(results[0]['city'])

Dhahran


# 4. Print the company established before 1950, which has the largest annual revenue.

In [154]:
query3 = {
    "founded":{
        '$lt': 1950,
    }
}

results4 = collection.find(query3).sort('revenue',-1).limit(1)
print(results4[0]['company'])


Toyota Motor


# 5. Print the total revenue and the total number of employees of the e-commerce companies.

In [152]:
pipeline2 = [
{'$match': {'description':{'$regex':'e-commerce',
                           "$options": 'i'}}},
    {'$group': {'_id': None, 'revenue_sum':{'$sum': "$revenue"}, 'emp_sum': {'$sum': '$employees'}}}
]
list(collection.aggregate(pipeline2))

[{'_id': None, 'revenue_sum': 579.31, 'emp_sum': 1859462}]